# Import des outils / jeu de données

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import prince
import seaborn as sns
from sklearn.cluster import (
    DBSCAN,
    OPTICS,
    AffinityPropagation,
    AgglomerativeClustering,
    KMeans,
    MeanShift,
)
from sklearn.compose import ColumnTransformer
from sklearn.metrics import (
    calinski_harabasz_score,
    davies_bouldin_score,
    silhouette_score,
)
from sklearn.mixture import GaussianMixture
from sklearn.preprocessing import RobustScaler, StandardScaler

from clustering import initiate_cluster_models
from constants import var_categoriques, var_numeriques

In [ ]:
SEED = 0

In [ ]:
np.random.seed(SEED)
sns.set_theme()

In [ ]:
df = pd.read_csv(
    "../data/data-cleaned-feature-engineering.csv",
    sep=",",
    index_col="ID",
    parse_dates=True,
)

In [ ]:
composantes_acp = pd.read_csv("../data/composantes_acp.csv", index_col="ID")
composantes_acm = pd.read_csv("../data/composantes_acm.csv", index_col="ID")

## Variables globales

In [ ]:
var_categoriques_extra = ["NbAcceptedCampaigns", "HasAcceptedCampaigns", "NbChildren"]

var_categoriques_fe = var_categoriques + var_categoriques_extra

# Clustering

## Préparation des données

Nous commencer par fusionner les variables quantitatives et les coordonnées des individus dans l'ACM.

In [ ]:
X_clust = pd.concat((df[var_numeriques], composantes_acm), axis=1)

In [ ]:
X_clust.head()

In [ ]:
preprocessor = ColumnTransformer(
    remainder="passthrough",
    transformers=[
        ("scaler", RobustScaler(), var_numeriques),
    ],
)

In [ ]:
scaler = RobustScaler()
df_apres_scale = pd.DataFrame(
    preprocessor.fit_transform(X_clust),
    columns=X_clust.columns,
    index=df.index,
)

In [ ]:
df_apres_scale.head()

In [ ]:
df_avec_clusters = df_apres_scale.copy()

## Différents algorithmes de clustering

Nous choisissons de tester 2 types de modèles de clustering :
1) les modèles à choix du nombre de clusters
2) les modèles qui décident du nombre de clusters

Cela nous permettra de comparer le nombre de clusters donné par les seconds algorithmes.

Pour les modèles pour lesquels il faut choisir le nombre de clusters, nous décidons de tester des clusters de taille 2 à 5 (inclus), car un trop grand nombre de clusters serait plus difficile à interpréter pour l'équipe marketing dans un premier temps.

**Tableau.** Méthodologie de clustering

|:----------------------------|:----|
| Algorithmes | Avec choix du nombre de clusters (entre 2 et 5)<br>Sans choix du nombre de clusters |
| Critères de sélection | Répartition des clusters<br>Métriques de clusters<br>Sélection manuelle des clusters via leur affichage |
| Métriques | Score Silhouette (entre -1 et 1, proche de 1 = meilleurs clusters)<br>Calinski-Harabasz (entre 0 et $+\infty$ plus grand = meilleure séparation)<br>Davies-Bouldin (entre 0 et $+\infty$, proche de 0 = meilleurs clusters) |
| Affichage des clusters | Sur les axes d'ACP 1-4<br>Sur les axes d'ACM 1-4<br>En fonction des variables quantitatives<br>En fonction des variables qualitatives |


**Tableau.** Algorithmes de clustering testés

| Choix du nombre de clusters | Algorithmes                                                                                                                                    |
|:----------------------------|:-----------------------------------------------------------------------------------------------------------------------------------------------|
| Avec                        | KMeans<br/>Mélange Gaussien (GMM)<br/>Classification Ascendante Hiérarchique (CAH)<br/>(méthode de Ward, single/complete/average linkage) |
| Sans                        | OPTICS<br/>Mean Shift<br/>Propagation d'affinité (Affinity Propagation)                                                                        |

In [ ]:
NB_CLUSTER_MIN = 2
NB_CLUSTER_MAX = 6  # non inclus

In [ ]:
model_clusters = initiate_cluster_models(
    NB_CLUSTER_MIN,
    NB_CLUSTER_MAX,
    SEED,
)

In [ ]:
a = GaussianMixture()

In [ ]:
isinstance(a, GaussianMixture)

In [ ]:
cluster_metrics = []

for (model_name, model) in model_clusters.items():
    if isinstance(model, GaussianMixture):  # cas particulier du mélange gaussien
        df_avec_clusters[model_name] = model.fit_predict(df_apres_scale)
    else:
        model.fit(df_apres_scale)
        df_avec_clusters[model_name] = model.labels_

    df_avec_clusters[model_name] = pd.Categorical(
        df_avec_clusters[model_name].astype(str)
    )

    nb_clusters = df_avec_clusters[model_name].nunique()

    repartition = list(
        df_avec_clusters[model_name].value_counts(normalize=True).round(2).astype(str)
    )  # todo: enlever astype(str) si ça sert à rien (tester)

    cluster_metrics.append(
        [
            model_name,
            nb_clusters,
            " | ".join(repartition),
            silhouette_score(
                df_apres_scale, df_avec_clusters[model_name], random_state=SEED
            ),  # proche de 1 = mieux
            calinski_harabasz_score(
                df_apres_scale,
                df_avec_clusters[model_name],
            ),  # plus élevé, mieux c'est
            davies_bouldin_score(
                df_apres_scale, df_avec_clusters[model_name]
            ),  # proche de 0 = mieux
        ]
    )

In [ ]:
pd.DataFrame(
    cluster_metrics,
    columns=[
        "Algorithme de clustering",
        "Nombre de clusters",
        "Répartition",
        "Silhouette",
        "Calinski-Harabasz",
        "Davies-Bouldin",
    ],
)

Clusters sélectionnés :
- KMeans 2
- GMM 2
- CAH (Ward 2)

Nous avons aussi étudié certains clusters avec 3 groupes, qui nous ont permis d'identifier certains individus, mais qui ne sont pas aussi intéressants et utilisables que les clusters avec 2 groupes.

## Visualisation

In [ ]:
def affiche_taille_clusters(nom_cluster):
    plt.title("Taille des clusters")
    sns.histplot(df_avec_clusters[nom_cluster], shrink=0.5)

    plt.show()

In [ ]:
def affiche_clusters_acp(nom_cluster):
    _, ax = plt.subplots(1, 2, figsize=(12, 5))

    ax[0].set_title("Clusters sur les axes d'ACP 1-2")
    ax[1].set_title("Clusters sur les axes d'ACP 3-4")

    sns.scatterplot(
        composantes_acp,
        x="ACP1",
        y="ACP2",
        hue=df_avec_clusters[nom_cluster],
        alpha=0.8,
        ax=ax[0],
    )
    sns.scatterplot(
        composantes_acp,
        x="ACP3",
        y="ACP4",
        hue=df_avec_clusters[nom_cluster],
        alpha=0.8,
        ax=ax[1],
    )

    plt.show()

In [ ]:
def affiche_clusters_acm(nom_cluster):
    _, ax = plt.subplots(1, 2, figsize=(12, 5))

    ax[0].set_title("Clusters sur les axes d'ACM 1-2")
    ax[1].set_title("Clusters sur les axes d'ACM 3-4")

    sns.scatterplot(
        composantes_acm,
        x="ACM1",
        y="ACM2",
        hue=df_avec_clusters[nom_cluster],
        alpha=0.8,
        ax=ax[0],
    )

    sns.scatterplot(
        composantes_acm,
        x="ACM3",
        y="ACM4",
        hue=df_avec_clusters[nom_cluster],
        alpha=0.8,
        ax=ax[1],
    )

    plt.show()

In [ ]:
def affiche_clusters_var_quanti(nom_cluster):
    """Affiche les variables quantitatives en fonction des clusters."""
    for var in var_numeriques:
        _, ax = plt.subplots(1, 2, figsize=(10, 3))

        sns.boxplot(
            x=df[var],
            y=df_avec_clusters[nom_cluster],
            width=0.25,
            ax=ax[0],
        )

        sns.histplot(
            x=df[var],
            kde=True,
            ax=ax[1],
            hue=df_avec_clusters[nom_cluster],
            stat="probability",
            common_norm=False,
        )

        plt.show()

In [ ]:
def affiche_clusters_var_quali(nom_cluster):
    """Affiche les variables qualitatives en fonction des clusters et vice-versa."""
    for var in var_categoriques_fe:
        _, ax = plt.subplots(1, 2, figsize=(10, 4))

        sns.histplot(
            x=df[var].astype(str),
            ax=ax[0],
            hue=df_avec_clusters[nom_cluster],
            multiple="dodge",
            shrink=0.5,
            common_norm=True,
        )

        sns.histplot(
            hue=df[var].astype(str),
            ax=ax[1],
            x=df_avec_clusters[nom_cluster],
            multiple="dodge",
            shrink=0.5,
            common_norm=True,
        )

        plt.show()

In [ ]:
def affiche_clusters(nom_cluster):
    """Affiche les variables en fonction des clusters."""
    affiche_taille_clusters(nom_cluster)
    affiche_clusters_acp(nom_cluster)
    affiche_clusters_acm(nom_cluster)

    affiche_clusters_var_quanti(nom_cluster)
    affiche_clusters_var_quali(nom_cluster)

In [ ]:
affiche_clusters("KMeans2")

In [ ]:
affiche_clusters("GMM2")

In [ ]:
affiche_clusters("CAH (Ward) 2")

# Conclusion

**Tableau.** Description des clients types

| Profil                            | Proportion | Education         | Revenu          | Campagnes<br>acceptées | Enfants                          | Dépenses    |  Année de<br/>naissance | Site Internet |
|:----------------------------------|-----------:|:------------------|:----------------|-----------------------:|:---------------------------------|:------------|-------------------:|:--------------------|
| Clients qui achètent              |        30% | Bac - doctorat    | Élevé           |                    0-4 | 0 bas-âge <br/> 0-1 ado          | Élevées     |               1970 | Peu de visites      |
| Clients qui n'achètent pas ou peu |  68% - 70% | Brevet - doctorat | Moyen           |                    0-1 | 0-3 enfants<br/>(bas-âge et ado) | Peu élevées |               1970 | Beaucoup de visites |
| Clients qui n'achètent pas (n=3)  |         2% | Brevet            | Le plus faible  |                      0 | 0-1 bas-âge <br/> 0 ado          | Aucune      |               1980 | Beaucoup de visites |

Notons aussi que parmi les clients qui achètent, la proportion d'acceptation des campagnes est beaucoup plus élevée.

# Pour aller plus loin

- tester la stabilité des clusters (ici, l'initialisation des algorithmes a un impact significatif sur les clusters trouvés)
- tester les différents paramètres de chacun des algorithmes de clusters pour comparaison
- tester les algorithmes de clustering sur différents sous-ensembles de variables pour exhiber différents groupes

# Sauvegarde des données

In [ ]:
# todo: sauvegarder les clusters